# Deux labo : résultat contradictoire ?

In [175]:
#importation package
import pandas as pd
import duckdb

#config affichage
pd.set_option("display.max_columns", None)  # Affiche toutes les colonnes sans limitation
pd.set_option("display.max_colwidth", None)  # Affiche la largeur complète des colonnes sans couper le texte
pd.set_option("display.expand_frame_repr", False)  # Empêche le repliement des colonnes lors de l'affichage du DataFrame

# Connexion
from pipelines.tasks.config.common import DUCKDB_FILE
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)


## Identifier : plusieurs analyses pour un prélévement  ##

un prélévement fait l'objet d'un ou plusieurs analyses pour un seul paramètre  
eg :  
- referenceprel 00100139398, trois analyses différents pour le paramètre ASP  
- referenceprel 00100139917, trois analyses pour le paramètre ODQ  
- ...

In [176]:
# prélévement avec plusieurs analyse d'un même paramètre
query_plusieurs_analyses = """ 
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux
 """
plusieurs_analyses = con.sql(query_plusieurs_analyses).df()
plusieurs_analyses

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,referenceprel,cdparametresiseeaux,nbanalyse
0,00100139398,ASP,3
1,00100139398,ODQ,3
2,00100139917,ASP,3
3,00100139917,ODQ,3
4,00100139918,ASP,3
...,...,...,...
104195,97600032328,TURBNFU,2
104196,97600032332,PH,2
104197,97600032332,TURBNFU,2
104198,97600032333,PH,2


## Explo global : plusieurs analyses - résultats contradictoires ##

Un prélévement peut faire l'objet de plusieurs analyses, sur terrain et/ou en laboratoire.  
Les résultats peuvent être contradictoire pour le même paramètre analysé

##### Un paramètre - plusieurs analyses #####

In [189]:
# edc_resultat avec plusieurs analyse d'un paramètre
query_resultat_plusieurs_analyses = """ 
WITH
plusieurs_resultats AS (
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux)
,
resultat_diff AS (
SELECT *,
LAG(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_1, 
valtraduite AS valtraduite_0, 
LEAD(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux) AS valtraduite_2,
FROM edc_resultats
WHERE referenceprel IN (SELECT DISTINCT referenceprel
                        FROM plusieurs_resultats)
ORDER BY referenceprel,cdparametresiseeaux)

SELECT *
FROM resultat_diff
WHERE  valtraduite_1 IS NOT NULL OR valtraduite_2 IS NOT NULL
ORDER BY referenceprel, cdparametresiseeaux

 """
resultat_plusieurs_analyses = con.sql(query_resultat_plusieurs_analyses).df()
resultat_plusieurs_analyses

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime,valtraduite_1,valtraduite_0,valtraduite_2
0,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147233,2024,2025-02-28,20250203-075802,0.00,0.00,NaN
1,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147232,2024,2025-02-28,20250203-075802,NaN,0.00,0.00
2,001,00100138503,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,00100147232,2024,2025-02-28,20250203-075802,NaN,0.00,0.00
3,001,00100138503,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,00100147233,2024,2025-02-28,20250203-075802,0.00,0.00,NaN
4,001,00100139037,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147812,2024,2025-02-28,20250203-075802,NaN,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207292,976,97600032332,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,N,L,"0,26",NFU,232,<=1 NFU,"<=0,5 NFU",0.26,None,97600042530,2024,2025-02-28,20250203-075802,NaN,0.26,0.26
207293,976,97600032333,PH,1302,PH,pH,None,N,L,"7,9",unité pH,264,None,">=6,5 et <=9 unité pH",7.90,None,97600042532,2024,2025-02-28,20250203-075802,7.50,7.90,NaN
207294,976,97600032333,PH,1302,PH,pH,None,N,T,"7,5",unité pH,264,None,">=6,5 et <=9 unité pH",7.50,None,None,2024,2025-02-28,20250203-075802,NaN,7.50,7.90
207295,976,97600032333,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,N,L,"0,48",NFU,232,<=1 NFU,"<=0,5 NFU",0.48,None,97600042532,2024,2025-02-28,20250203-075802,NaN,0.48,0.73


#### Plusieurs analyses - résultats contradictoires ####

2 cas :  
- le prélévement n'a pas été analysé sur terrain mais affiche une valtraduite
- le prélévement a été analysé sur terrain, ensuite une autre analyse en laboratoire (résultats différents)

eg : 
Pour les qualitparam qualitatif (O)

- referenceprel 00100139040 -  rqana : Aspect normal >> Aspect anormal  - valtraduite : 0.0 >> 1.0 - insituana   L / L 
- referenceprel 00400142921 -  rqana : Aucun changement anormal >> Changement anormal - valtraduite : 0.0 >> 1.0 - insituana  T / L 
- referenceprel 06800170979 -  rqana : Non mesuré	>> Aucun changement anormal - valtraduite : 2.0 >> 0.0 - insituana T / L

Pour les qualitparam quantitatif (N)
- referenceprel 00100142701 - rqana : 0,005 >> 0,004 - valtraduite : 0.005 >> 0.004 - insituana L / L
- referenceprel 97600032333 - rqana : 0.730 >> 0.480 - valtraduite : 0.730 >> 0.480 - insituana T / L




In [ ]:
#Explo global resultats différent
query_explo_global_resultat_diff = """ 

SELECT referenceprel, cdparametresiseeaux, libmajparametre, insituana, cdunitereference, limitequal, refqual, rqana, qualitparam, valtraduite_1, valtraduite_0, valtraduite_2, 
FROM resultat_plusieurs_analyses
WHERE (valtraduite_1 != valtraduite_0 OR valtraduite_0 != valtraduite_2)
ORDER BY referenceprel, cdparametresiseeaux


"""
explo_global_resultat_diff = con.sql(query_explo_global_resultat_diff).df()
explo_global_resultat_diff

,referenceprel,cdparametresiseeaux,libmajparametre,insituana,cdunitereference,limitequal,refqual,rqana,qualitparam,valtraduite_1,valtraduite_0,valtraduite_2
0,00100139040,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect normal,O,NaN,0.00,1.00
1,00100139040,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect anormal,O,0.00,1.00,NaN
2,00100139188,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect normal,O,NaN,0.00,1.00
3,00100139188,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect anormal,O,0.00,1.00,NaN
4,00100139516,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect normal,O,1.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
171941,97600032332,PH,PH,L,264,None,">=6,5 et <=9 unité pH","7,6",N,NaN,7.60,7.66
171942,97600032333,PH,PH,L,264,None,">=6,5 et <=9 unité pH","7,9",N,7.50,7.90,NaN
171943,97600032333,PH,PH,T,264,None,">=6,5 et <=9 unité pH","7,5",N,NaN,7.50,7.90
171944,97600032333,TURBNFU,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,T,232,<=1 NFU,"<=0,5 NFU","0,73",N,0.48,0.73,NaN


In [ ]:
#Explo global resultats différent - types prélévements
query_type_referenceprel = """ 

SELECT *
FROM explo_global_resultat_diff
WHERE referenceprel IN ('00100139040','00400142921','06800170979','00100142701','97600032333')
ORDER BY qualitparam DESC, referenceprel, cdparametresiseeaux

"""
type_referenceprel = con.sql(query_type_referenceprel).df()
type_referenceprel

,referenceprel,cdparametresiseeaux,libmajparametre,insituana,cdunitereference,limitequal,refqual,rqana,qualitparam,valtraduite_1,valtraduite_0,valtraduite_2
0,00100139040,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect normal,O,NaN,0.000,1.000
1,00100139040,ASP,ASPECT (QUALITATIF),L,X,None,None,Aspect anormal,O,0.000,1.000,NaN
2,00400142921,COULQ,COULEUR (QUALITATIF),T,X,None,None,Aucun changement anormal,O,1.000,0.000,NaN
3,00400142921,COULQ,COULEUR (QUALITATIF),L,X,None,None,Changement anormal,O,NaN,1.000,0.000
4,06800170979,SAVQ,SAVEUR (QUALITATIF),L,X,None,None,Aucun changement anormal,O,NaN,0.000,2.000
5,06800170979,SAVQ,SAVEUR (QUALITATIF),T,X,None,None,Non mesuré,O,0.000,2.000,NaN
6,00100142701,CHRYS,CHRYSÈNE,L,133,None,None,"0,004",N,NaN,0.004,0.005
7,00100142701,CHRYS,CHRYSÈNE,L,133,None,None,"0,005",N,0.004,0.005,NaN
8,00100142701,FLUORA,FLUORANTHÈNE *,L,133,None,None,"0,083",N,NaN,0.083,0.092
9,00100142701,FLUORA,FLUORANTHÈNE *,L,133,None,None,"0,092",N,0.083,0.092,NaN


#### Explo insituana ( T - L )

Les différences de résultats concernent les analyses :  
-  Terrain vs  Labo
-  Labo vs  Labo

In [ ]:
#Explo global type d'analyse (Labo / terrain)
query_type_analyse = """ 
WITH
type_insituana AS (
SELECT referenceprel, cdparametresiseeaux, valtraduite_1, valtraduite_0, valtraduite_2, 
LAG(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS insituana_1, 
insituana AS insituana_0, 
LEAD(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS insituana_2,
FROM resultat_plusieurs_analyses
ORDER BY referenceprel, cdparametresiseeaux)


SELECT DISTINCT insituana_1, insituana_0, insituana_2
FROM type_insituana
WHERE insituana_1 IS NOT NULL OR insituana_2 IS NOT NULL
ORDER BY insituana_1


"""
type_analyse = con.sql(query_type_analyse).df()
type_analyse

,insituana_1,insituana_0,insituana_2
0,L,T,None
1,L,L,None
2,L,L,L
3,T,L,None
4,None,L,T
5,None,T,L
6,None,L,L


### Les paramètres concerné par plusieurs analyses

Sur l'ensemble des paramètres, 26% font l'objet de plusieurs analyses

In [ ]:
query_paramtètre = """  
WITH 
paramètre_plusieurs_resultats AS (
SELECT DISTINCT cdparametresiseeaux
FROM plusieurs_analyses )
,
paramètre_global AS (
SELECT DISTINCT cdparametresiseeaux
FROM edc_resultats
)
,
resultat_diff AS (
SELECT DISTINCT cdparametresiseeaux
FROM explo_global_resultat_diff
)

SELECT COUNT(DISTINCT pps.cdparametresiseeaux) AS nbparam, COUNT (DISTINCT prg.cdparametresiseeaux) AS nbparamtotal, COUNT (DISTINCT rtd.cdparametresiseeaux) AS nbparamresultatdiff,
    ROUND (nbparam / nbparamtotal * 100 , 2) AS prctglob,ROUND (nbparamresultatdiff / nbparam * 100, 2) AS prctplsresultat
FROM paramètre_plusieurs_resultats AS pps
CROSS JOIN paramètre_global AS prg
CROSS JOIN resultat_diff AS rtd
"""
paramètre = con.sql(query_paramtètre).df()
paramètre



,nbparam,nbparamtotal,nbparamresultatdiff,prctglob,prctplsresultat
0,349,1338,35,26.08,10.03


## Conclusion ##

Un paramètre issu d'un même prélévement peut faire l'objet de plusieurs analyses dont une première analyse sur terrain et plusieurs analyses en laboratoires.  

Les analyses sur terrain (insituana = T ) ne possède pas de référence d'analyse 'referenceanl'  

Les contradictoires sont présents sur des configuration :  
    - analyse 1 sur Terrain - analyse 2 en Laboratoire ( insituana T - L )  
    - analyse 1 en Laboratoire - analyse 2 en Laboratoire (ou plus) ( insituana L - L )  

Dans la table edc_resultats, 26 % des paramètres ont été analysés plus d'une fois dont 10 % présentents des résultats contradictoires  

Les informations de la table edc_resultats ne permet pas d'identifier les raisons qui amène à ces contradictions ( à creuser ?? ) :  
les lignes concernées présente des valeurs identiques pour chaque champ à part :  referenceanl - rqana - valtraduite  



